### Persistance
Since Redis stores data in memory, it needs to write it to disk in order to recover data in case of crash or restart. The data transferred to the disc must be an exact copy of data in memory. But how do we achieve that? Suppose there are $n$ key value pairs at time $T_1$ when the data copy was started. Now during the copy process one of the key value pair can be modified. So there may be situation like $a$ amount of key values are same as they were at $T_1$, but $b$ number of key value pairs got updated and reflect the value they have at $T_2$.  

![Copy problem](https://i.imgur.com/y6Ac0Uw.png)

Possible solution:
- lock the entire database during the copy process
- make use of `fork`

The fork system call spawns a new child process and make two identical copies of address spaces, one for the parent and the other for the child. Since both the parent and child process now share the same physical memory block this is not an expensive operation.  
![Fork](https://i.imgur.com/W8i80Mb.png)  

Now when an update is made to the existing key value pair during copy, the updated key value pair gets a new memory location. Using this method at the maximum, double memory will be required.  
![Update](https://i.imgur.com/rdxJ3qb.png)  

Another thing to consider is how data from memory is written to disk? It happens in multiple steps:  
![Save](https://i.imgur.com/Pltcc6x.png)

`fsync` is a system call which transfers data from kernel buffer to disk cache. The final step is writing data to the disk, which we don't have control over. Fsync is expensive operation and it will block the process for all the time needed to complete the write. To conclude:
- Data written to kernel buffers using the write system call (or equivalent) gives us data safety against process failure (Redis crashes).
- Data committed to the disk using the fsync system call (or equivalent) gives us, virtually, data safety against complete system failure like a power outage.

Redis does disk persistance in two ways:
- **RDB**
- **AOF**

### Redis Database File
It is a compacted snapshot of database at a particular point in time. Redis allows configuring specific time duration after which snapshot will be taken. Problem is if Redis crashes during two snapshot, the data added/modified during that time delta will be lost. Configuration related to snapshotting is stored in Redis configuration file:

```
# In the example below the behavior will be to save:
# after 900 sec (15 min) if at least 1 key changed
# after 300 sec (5 min) if at least 10 keys changed
# after 60 sec if at least 10000 keys changed

save 900 1
save 300 10
save 60 10000

# By default Redis will stop accepting writes if RDB snapshots are enabled
# (at least one save point) and the latest background save failed.
# This will make the user aware (in a hard way) that data is not persisting
# on disk properly, otherwise chances are that no one will notice and some
# disaster will happen.

stop-writes-on-bgsave-error yes

# Compress data?
rdbcompression yes

# The filename where to dump the DB
dbfilename dump.rdb

# Where to save the dump?
dir ./
```

### Append Only File
Alternate persistance mode. In this mode all the commands (only if they have some effect on actual data) coming to Redis is first written to a file. Now in case of abrupt power loss or crash, the file is played back to bring system back to its original state. There is a possibility that the aof file becomes too big. Redis will simply rewrite aof file from scratch in a temporary file by directly accessing data in memory, so that Redis can create the shortest AOF that is possible to generate. There are three different configurations that will change the exact behavior of how Redis writes to disk:  

**appendfsync no:** Redis does not perform fsync call, data will be committed to disk at kernel's wish, that is, every 30 seconds in most Linux systems.  

**appendfsync everysec:** data will be both written to the file using write and flushed from the kernel to the disk using fsync one time every second.  

**appendfsync always:** data is both written to the file and synched on disk using fsync(2) before an acknowledge is returned to the client. Highest durability level.  

Sample config:
```
appendonly no

# Filename?
appendfilename "appendonly.aof"

# Mode
appendfsync everysec
```